<a href="https://colab.research.google.com/github/ReillyOareVT/ML_in_WR/blob/main/CI_PI_lecture_3_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lecture 3, 01/28/2025

---

Today, we will deal with generating confidence and prediction intervals on sample statistics and observations.

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns


# Parametric confidence intervals for the mean
### We sample chloride from 10 locations in an aquifer
The true mean (μ) of 5.0 mg/L and the standard deviation (σ) is 1.0 mg/L.

We repeat this sampling 10 times to demonstrate uncertainty in sampling.

We want to be 80% ($α$ = 0.20) sure that our sample contains the true mean.

Recall that the confidence interval is computed as:

$x̄ ± z \frac{s}{\sqrt{n}}$

, where z is the z-score. This assumes symmetric confidence intervals.

In [ ]:
# Write out the model parameters
population_mean = 5
population_std = 1
sample_size = 10
confidence_level = 0.80
num_experiments = 10

# Arrays to store results
sample_means = []
conf_intervals = []

# Arrays to store results
sample_means = []
conf_intervals = []
all_samples = []

# Conduct replicate experiments
for _ in range(num_experiments):
    # Draw samples from the normal distribution
    samples = np.random.normal(population_mean, population_std, sample_size)
    all_samples.append(samples)

    # Calculate sample mean
    sample_mean = np.mean(samples)
    sample_means.append(sample_mean)

    # Calculate standard error of the mean
    sem = stats.sem(samples)

    # Calculate confidence interval
    margin_of_error = sem * stats.t.ppf((1 + confidence_level) / 2, sample_size - 1)
    conf_interval = (sample_mean - margin_of_error, sample_mean + margin_of_error)
    conf_intervals.append(conf_interval)

### Data visualization
Show the results of the true distribution, true mean, and our estimated confidence intervals from our samples.

In [ ]:
# Visualization
fig, ax = plt.subplots(figsize=(6, 8))
# remove borders around figure for cleaner look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

# Plot normal distribution density at the top
x = np.linspace(population_mean - 3*population_std, population_mean + 3*population_std, 1000)
y = stats.norm.pdf(x, population_mean, population_std)
ax.plot(x, 9 + y * 2.5, color='black')  # Scale density for visibility
ax.axvline(population_mean, color='black', linestyle='dashed', linewidth=1.5)
ax.text(population_mean, num_experiments + 0.22, '$\mu$', ha='center', fontsize=9, backgroundcolor='w', color='k')

# Plot confidence intervals and samples
for i, (mean, (lower, upper), samples) in enumerate(zip(sample_means, conf_intervals, all_samples)):
    color = 'blue' if lower <= population_mean <= upper else 'red'
    ax.plot([lower, upper], [i, i], color=color, marker='none', markersize=5, zorder = 5)
    ax.scatter(sample_means[i], [i], color='black', alpha=1.0, s=15, zorder = 10)
    ax.scatter(samples, [i] * len(samples), color='black', alpha=0.2, s=10, zorder = 0)

ax.set_yticks(range(num_experiments))
ax.set_yticklabels(range(1, num_experiments + 1))
ax.set_xlabel('Sample Mean')
ax.set_ylabel('Experiment Number')
ax.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

# Non-parametric confidence intervals for the median
### We sample nitrate 25 times from a river
The values (mg-N/L)are reported in ascending order below:

`1.3, 1.5, 1.8, 2.6, 2.8, 3.5, 4.0, 4.8, 8.0, 9.5, 12, 14, 19, 23, 41, 80, 100, 110, 120, 190, 240, 250, 300, 340, 580`

First, it may help to show the distribution of the data.

In [ ]:
# Define the array of data
data = [1.3, 1.5, 1.8, 2.6, 2.8, 3.5, 4.0, 4.8, 8.0, 9.5, 12, 14, 19, 23, 41, 80, 100, 110, 120, 190, 240, 250, 300, 340, 580]

# Calculate the natural log of the array to show sublte changes on the skewed part of the distribution
log_data = np.log(data)

# Create the figure and subplots
fig, axes = plt.subplots(1, 2, figsize=(6, 3))

# Boxplot for original data
axes[0].boxplot(data, vert=True)
axes[0].set_title("Boxplot of Original Data")
axes[0].set_ylabel("Concentration (mg-N/L)")

# Boxplot for log-transformed data
axes[1].boxplot(log_data, vert=True)
axes[1].set_title("Boxplot of Log-Transformed Data")

# Adjust layout and show plot
plt.tight_layout()
plt.show()

print('The sample median is', np.median(data), 'mg-N/L')


## Binomial distribution approach
Let's calculate the 95% confidence interval estimate of the median ($c_{50}$) using the binomial distribution approach.

In [ ]:
# Parameters of binomial
p = 0.5  # Probability which equals the median (50th percentile)
n = 25   # Size of dataset
prob = [0.025, 0.975]  # Desired bounds of cumulative probability (2.5th and 97.5th, which contain 95%)

# Calculate the index of the confidence intervals
bin_index = stats.binom.ppf(prob, n, p)

# Retreive the data value corresponding to the indices
lowCI = int(bin_index[0]); highCI = int(bin_index[1]);

print('The 95% confidence intervals for the median are:', data[lowCI], 'and', data[highCI], 'mg-N/L')


## Bootstrapping approach
Let's calculate the 95% confidence interval estimate of the median ($c_{50}$) using the bootstrapping approach.

In [ ]:
# Perform bootstrap sampling to calculate the 95% confidence interval for the median
confidence_level = 0.95
res = stats.bootstrap((data,), np.median, confidence_level=confidence_level, n_resamples=1000, method='percentile')

ci_lower, ci_upper = res.confidence_interval
median_estimate = np.median(data)

print(f"95% confidence interval for the median: {ci_lower:.0f} to {ci_upper:.0f} mg-N/L")

### Data visualization
Show the results of the process of bootstrapping and the final estimate

In [ ]:
# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(6, 6), gridspec_kw={'height_ratios': [2, 1]})

# Determine x-axis limits
x_min = min(min(data), min(bootstrap_samples))
x_max = max(max(data), max(bootstrap_samples))

# Top subplot: Scatter plot of 10 bootstrap realizations, evenly spaced vertically
y_positions = np.linspace(1, 10, 10)
for y_pos in y_positions:
    sample = np.random.choice(data, size=len(data), replace=True)
    sample_median = np.median(sample)
    axes[0].scatter(sample, np.full_like(sample, y_pos), edgecolor='black', facecolors='none', alpha=0.5)
    axes[0].axvline(sample_median, color='red', linestyle='--', alpha=0.5)

axes[0].axvline(np.median(data), color='red', linestyle='--', linewidth=2)
axes[0].set_xticks([])
axes[0].set_yticks([])
axes[0].set_title("Example Bootstraps w/ Sample Medians")
axes[0].set_xlim(x_min, x_max)

# Bottom subplot: KDE of bootstrap samples using seaborn
sns.kdeplot(bootstrap_samples, ax=axes[1], color='black', linewidth=2, linestyle=':', label='KDE')

# Confidence interval
axes[1].errorbar(np.median(data), 0.04, xerr=[[np.median(data) - ci_lower], [ci_upper - np.median(data)]], fmt='o', color='black',label='95% CI')

# Formatting
axes[1].set_xticks(np.linspace(x_min, x_max, 6))
axes[1].set_yticks([])
axes[1].set_title("Kernel Density of Bootstrap Medians")
axes[1].set_xlabel("Median Values")
axes[1].set_xlim(x_min, x_max)
axes[1].legend(loc='upper right')

plt.tight_layout()
plt.show()